# Car Detection

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving costco_carpark.tif to costco_carpark.tif


In [2]:
raster_path = "/content/costco_carpark.tif"
raster_url  = raster_path

In [3]:
pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

## Import libraries

In [4]:
import geoai

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [5]:
raster_path = "/content/costco_carpark.tif"

In [6]:
raster_url  = raster_path

## Visualize the image

In [7]:
geoai.view_raster(raster_url)

## Initialize the model

In [8]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...


car_detection_usa.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/089548329c81f128fa12576663e7abdedb5cfa0e/car_detection_usa.pth
Model loaded successfully


## Extract cars

Extract cars from the image using the model and save the output image.

In [17]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.5,
    mask_threshold=0.7,
    overlap=0.25,
    chip_size=(400, 400),
)

Dataset initialized with 11 rows and 17 columns of chips
Image dimensions: 4846 x 3250 pixels
Chip size: 400 x 400 pixels
Overlap: 25.0% (stride_x=300, stride_y=300)
CRS: EPSG:6428
Processing raster with 47 batches


100%|██████████| 47/47 [23:14<00:00, 29.68s/it]


Masks with confidence values saved to cars_masks.tif


Convert the image masks to polygons and save the output GeoJSON file.

In [26]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=50,  # Adjusted to a more reasonable minimum
    max_object_area=5000, # Adjusted to a more reasonable maximum
)

Processing masks from: cars_masks.tif
Found 481 connected components
Using single-threaded processing, you can speed up processing by setting n_workers > 1


Processing components: 100%|██████████| 481/481 [00:14<00:00, 33.31it/s]

Saved 477 objects with confidence to cars.geojson
Total processing time: 14.77 seconds


## Add geometric properties

In [27]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [28]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter cars by area

In [29]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [24]:
len(gdf_filter)

7

## Visualiza final results

In [31]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [32]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)